In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
%autosave 10

import pickle
import sys, os
sys.setrecursionlimit(10000)


import gensim
import pandas as pd
import scipy as sp
import numpy as np
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

from sklearn.preprocessing import OneHotEncoder
from sklearn.cross_validation import train_test_split, ShuffleSplit
from sklearn.metrics import classification_report, precision_recall_fscore_support, accuracy_score
from sklearn.grid_search import GridSearchCV, ParameterGrid

from sklearn.externals.joblib import load, dump
from sklearn.metrics import f1_score, accuracy_score, recall_score, precision_score, confusion_matrix

Autosaving every 10 seconds


/home/sag/src_George/workspace/virtualenv/lib/python3.4/site-packages/gensim/utils.py:1015: UserWarning: Pattern library is not installed, lemmatization won't be available.
  warnings.warn("Pattern library is not installed, lemmatization won't be available.")
/home/sag/src_George/workspace/virtualenv/lib/python3.4/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/sag/src_George/workspace/virtualenv/lib/python3.4/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be

In [3]:
gpu_id='gpu2'
import os
os.environ["THEANO_FLAGS"] = "device="+gpu_id +",lib.cnmem=1"



from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers import Activation, Dense, Dropout, LSTM, Merge
from keras.layers.embeddings import Embedding
from keras.callbacks import EarlyStopping
from keras.wrappers.scikit_learn import KerasClassifier

Using Theano backend.
Using gpu device 2: Tesla K80 (CNMeM is enabled with initial size: 95.0% of memory, cuDNN 4007)


In [4]:
LEMMA_IND = 7
POS_IND = 9
SELECTED_FEATURES = "/home/sag/syntactic/src_putin/grant/data/processed/selected_features/"
BEST_MODEL = SELECTED_FEATURES+"model_without_w2v_init.pkl"
RESULTS = SELECTED_FEATURES+"results3.pkl"

In [5]:
matrix = np.load(SELECTED_FEATURES+"matrix.npy")
matrix = np.delete(matrix, (71079), axis=0)
#w2v_weights = np.load(SELECTED_FEATURES+"w2v_weights.npy")

target = OneHotEncoder(sparse=False).fit_transform(np.load(SELECTED_FEATURES+"y.npy").reshape(-1, 1) - 1)
target = np.delete(target, (71079), axis=0)
print(matrix.shape, target.shape)

#X_train, X_val, X_test, y_train, y_val, y_test = matrix_train_val_test_split(matrix, target, number_of_samples_to_use=10000)
X_train, X_val, X_test, y_train, y_val, y_test = matrix_train_val_test_split(matrix, target)

(1423937, 13, 12) (1423937, 125)


NameError: name 'matrix_train_val_test_split' is not defined

In [12]:
target[2]

array([ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])

In [5]:
# params = {
#     'lstm_layers' :[[450, 450, 450], [400, 400, 400]],
#     'dense_layers': [[100, 100], [100]],
#     'loss': ['categorical_crossentropy'],
#     'activation': ['srelu', 'prelu', 'elu'],
#     'optimizer': [Eve(), 'Adam'],
#     'metrics': [f1],
#     'dropout': [0.25]
# }

# fit_params = {
#     'nb_epoch': 100,
#     'batch_size': 3072,
#     'verbose': 2,
#     'callbacks': [EarlyStopping(monitor='val_loss', patience=5)],
#     'validation_data': [X_val, y_val]
# }

# branches = create_embedding_without_w2v_matrix(matrix)
# results, best_model, best_score = keras_gridsearch(
#                      create_stacked_lstm, \
#                      branches, \
#                      X_train, y_train, \
#                      X_val, y_val, \
#                      X_test, y_test, \
#                      params, fit_params, \
#                      save_best_model=BEST_MODEL,
#                      save_results=RESULTS, start=0, end=12
#                      )

In [6]:
# model = load_keras_model(BEST_MODEL)
# stats_val = get_stats(model, X_val, y_val)
# stats_test = get_stats(model, X_test, y_test)
# print(stats_val['f1'])
# print(stats_test['f1'])

In [7]:
param = {
    'lstm_layers' :[450, 450, 450],
    'dense_layers': [100],
    'loss': 'categorical_crossentropy',
    'activation': 'srelu', # 'prelu', 'elu',
    'optimizer': Eve(),
    'metrics': f1,
    'dropout': 0.25
}

fit_params = {
    'nb_epoch': 100,
    'batch_size': 3072,
    'verbose': 2,
    'callbacks': [EarlyStopping(monitor='val_loss', patience=5), 
                  FullModelCheckpoint(BEST_MODEL)],
    'validation_data': [X_val, y_val]
}

branches = create_embedding_without_w2v_matrix(matrix)

In [10]:
# keras_model = KerasClassifier(build_fn=create_stacked_lstm, branches=branches, **param)
# keras_model.fit(X_train, y_train, **fit_params)
# stats_val = get_stats(keras_model, X_val, y_val)
# stats_test = get_stats(keras_model, X_test, y_test)

In [14]:
# encoding=utf8  
import sys  


sys.setdefaultencoding('utf8')

AttributeError: 'module' object has no attribute 'setdefaultencoding'

In [15]:
def load_keras_model(file_path):
    from keras.models import model_from_json
    json_string, weigths = pickle.load(open(file_path, 'rb'))
    keras_model = model_from_json(json_string)
    keras_model.set_weights(weigths)
    return keras_model

In [16]:
model = load_keras_model(BEST_MODEL)

UnicodeDecodeError: 'ascii' codec can't decode byte 0x8f in position 2: ordinal not in range(128)

In [11]:
model = load_keras_model(BEST_MODEL)
#results = pickle.load(open(RESULTS, 'rb'))

stats_val = get_stats(model, X_val, y_val)
stats_test = get_stats(model, X_test, y_test)
print(stats_val['f1'])
print(stats_test['f1'])

142394/142394 [==============================] - 18s    
0.883240700321
0.871933583835
